make environment using r_spatial.yml in Spatial folder

In [ ]:
micromamba create -n r_spatial -f r_spatial.yml -c conda-forge --channel-priority strict

R

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("nnSVG")


In [ ]:
#locally
mamba activate r_spatial
rstudio #will open rstudio locally with mounts


Note reasoning for batches is in comparing_outliers.csv  
 - #0 manual processing in interactive R session on cluster 
 - #1 to process each sample individually via slurm submission - avoiding as it results in different numbers of rows in count matrix of each sample which prevents merging later
 - #2 to process in bulk up to or including nnsvg step  

**skip ahead to 2**


0. To manually run the preprocessing pipeline

In [ ]:

# #open interactive node session, in terminal connect to cluster then enter

srun --pty --job-name=interactive_r --mem=32G --cpus-per-task=4 --partition=epyc --time=024:00:00 bash 
#activate r environment with
micromambda activate r_env #or r_spatial
#open R
R
Sys.setenv(PATH = paste(Sys.getenv("PATH"), "/home/lythgo02/micromamba/envs/r_env", sep = ":"))
#then run the commands in 20240204_QC_final_workflow.rmd but you will need to adapt the code for the samples you want to process as logbatch mitochondrial filters and the arbitrary mitochondrial filters batch

1. to run by as individual samples, open ssh terminal and set of individual slurm jobs for each (using a different script for the log batch and the arbitrary batch)

In [ ]:
#1.0

cd /mnt/scratchc/fmlab/lythgo02/visium_data/
for SAMPLE in SITSA3 SITSB2 SITSB4 SITSC1 SITSC3 SITSD3 SITSE4 SITSF2 SITSF4 ; 
do sbatch 
--job-name=${SAMPLE} 
--output=logs/${SAMPLE}.out 
--error=logs/${SAMPLE}.err 
--time=100:00:00 
--mem=32G 
--cpus-per-task=8 
--partition=epyc 
--wrap="Rscript single_sample_process_logmito.r ${SAMPLE}"; 
done

In [ ]:
#1.2

cd /mnt/scratchc/fmlab/lythgo02/visium_data/

for SAMPLE in SITSA1 SITSA2 SITSA4 SITSB1 SITSB3 SITSC2 SITSC4 SITSD1 SITSD2 SITSD4 SITSE2 SITSG2 SITSH2 ; 
 do sbatch 
 --job-name=${SAMPLE} 
 --output=logs/${SAMPLE}.out 
 --error=logs/${SAMPLE}.err 
 --time=048:00:00 
 --mem=32G 
 --cpus-per-task=8 
 --partition=epyc 
 --wrap="Rscript single_sample_processing_arbitrary.r ${SAMPLE}"; 
 done

once they have finished running cd into the respectiv folders and run the following

In [ ]:
#1.3

cd logbatch 

for SAMPLE in post_gettophgvSITSA3 post_gettophgvSITSB2 post_gettophgvSITSB4 post_gettophgvSITSC1 post_gettophgvSITSC3 post_gettophgvSITSD3 post_gettophgvSITSE4 post_gettophgvSITSF2 post_gettophgvSITSF4; 
 do     sbatch --job-name=${SAMPLE}            
 --output=logs/${SAMPLE}.out            
 --error=logs/${SAMPLE}.err            
 --time=0100:00:00            
 --mem=32G            
 --cpus-per-task=8            
 --partition=epyc            
 --wrap="Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/logbatch/nnsvg_logbatch.r ${SAMPLE}"; 
 done

In [ ]:
#1.4

cd into arbitrary

for SAMPLE in post_gettophgvSITSA1 post_gettophgvSITSA2 post_gettophgvSITSA4 post_gettophgvSITSB1 post_gettophgvSITSB3 post_gettophgvSITSC2 post_gettophgvSITSC4 post_gettophgvSITSD1 post_gettophgvSITSD2 post_gettophgvSITSD4 post_gettophgvSITSE2 post_gettophgvSITSG2 post_gettophgvSITSH2;  
do     
sbatch 
--job-name=${SAMPLE}            
--output=logs/${SAMPLE}.out            
--error=logs/${SAMPLE}.err            
--time=0100:00:00            
--mem=32G            
--cpus-per-task=8            
--partition=epyc            
--wrap="Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/arbitrary/nnsvg_arbitrary.r ${SAMPLE}";
done

#1.5 then pick up after running nnsvg 
#open interactive r session within micromamba r_rnv

In [ ]:
#1.5

knitr::opts_chunk$set(echo = TRUE)
library(spatialLIBD)
library(nnSVG)
library(SpatialExperiment)  
library(scater)
library(AnnotationHub)
library(tidyverse)
library(ggspavis)
library(scran)
library(DT)
projDir = "/mnt/scratchc/fmlab/lythgo02/visium_data/"


load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/logbatch/nnsvg"
# List files in the directory
files <- list.files(path = load_all_data, pattern = "^nnsvg", full.names = TRUE)

# Print out the list of files
print(files)

#assign names so they are loaded into the spe with the correct identifiers
logbatch <- c("SITSA3", "SITSB2", "SITSB4", 
                 "SITSC1","SITSC3","SITSD3",
                 "SITSE4","SITSF2", "SITSF4")

names(files) <- logbatch
# Initialize a list to store each sample
spe_list <- list()

# Loop through each file and process it
for (sample_name in names(files)) {
  # Read the Visium data
  spe <- readRDS(files[sample_name])
    # Add unique barcodes
  spe$barcode <- rownames(colData(spe))
  spe$barcodeid <- gsub("-1$", paste0("-", sample_name), spe$barcode)
  rownames(colData(spe)) <- spe$barcodeid
  spe$sample_id <- sample_name

  colData(spe)$qc_mito <- colData(spe)$log_is_outlier_mad
  colData(spe) <- colData(spe)[ , !(colnames(colData(spe)) %in% c("log_mito_percent","log_median_mito_percent","log_mad_mito_percent","log_deviation_above_median","log_is_outlier_mad"))]

  # Store the processed sample in the list
  spe_list[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}

In [ ]:
#1.6 load arbitrary batch 

load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/arbitrary/nnsvg"
# List files in the directory
files <- list.files(path = load_all_data, pattern = "^nnsvg", full.names = TRUE)

# Print out the list of files
print(files)
 
#assign names so they are loaded into the spe with the correct identifiers
arbitrary <- c("SITSA1","SITSA2","SITSA4", "SITSB1", "SITSB3", 
                 "SITSC2","SITSC4","SITSD1","SITSD2", "SITSD4",
                 "SITSE2","SITSG2", "SITSH2")

names(files) <- arbitrary 

# Loop through each file and process it
for (sample_name in names(files)) {
 # Read the Visium data
  spe <- readRDS(files[sample_name])
    # Add unique barcodes
  spe$barcode <- rownames(colData(spe))
  spe$barcodeid <- gsub("-1$", paste0("-", sample_name), spe$barcode)
  rownames(colData(spe)) <- spe$barcodeid
  spe$sample_id <- sample_name
  
  # Store the processed sample in the list
  spe_list[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}


In [ ]:
#1.6 bind all into one spatial object 


    spe <- do.call(cbind, spe_list) #wont work because different row numbers # wont work if row numbers differ

2. To process all samples together in a slurm submission (either up to nnsvg or inc nnsvg)
#make sure you are updating and using the scratch scripts, there are also copies on mnt but outdated

1_sample_specific_filtering_wholeworkflow.r isn't actually the whole workflow as the end is commented out so that next stages are run via 2_end....sh

In [ ]:
#2.0

sbatch /mnt/scratchc/fmlab/lythgo02/visium_data/scripts/submit_spatial_scripts.sh

#which is 

#!/bin/bash

#SBATCH --error=nnSVG.err
#SBATCH --time=0200:00:00             
#SBATCH --cpus-per-task=32      
#SBATCH --mem=120G                   
#SBATCH --partition=epyc     


# Run the R script
#Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/scripts/1_sample_specific_filtering_wholeworkflow.r 

In [ ]:
#there is also a script called 2_end_stage_hvg_nnsvg_singlesamplejob.r which is the end section of the workflow to submit single jobs for each sample for hvg and nnsvg

1_sample_specific_filtering_wholeworkflow.r runs the following steps:
Workflow Summary:


 - Load Visium spatial transcriptomics data per sample.
 - Add unique barcodes to each spot for tracking
 - Combine all samples into one SpatialExperiment (SPE) object.
 - Gene Annotationvia Ensembl
 - Filter out genes with zero counts.
 - Spot Filtering and QC:
    -  Keep only spots overlaying tissue.
 - Calculate per-spot QC metrics:
     - Number of detected genes,
     - Total UMIs (library size),
     - Percentage of mitochondrial UMIs.
 - Generate QC summary tables and save intermediate results.
 - Split cohort into log and arbitrary:
     - A few of he samples have high mito content so arbitrary threshold across all doesn't work, if we expect some mitochondrial content due to systematic/technical processes then we would expect this to show up fairly uniformly across samples. For these samples I apply a sample specific threshold. Exclude spots that are more than a particular standard deviation away from the mean. 
     - Mitochondrial % filter:
        -  In the “log” cohort, uses a sample-specific, robust outlier detection method on the log-transformed mitochondrial percentages (median ± MAD).
        -  In the “arbitrary” cohort, applies a fixed, arbitrary threshold (e.g., >20%).
    -  Library size and gene detection filters: use fixed thresholds for all samples regardless of cohort.
         - Library Size QC: Apply a threshold (e.g., <300 UMIs) to flag low-quality spots.
          -  Gene Detection QC: threshold (e.g., <300 detected genes) to flag poor spots.
 - Save QC-filtered SPE objects and plots at:
  - visium_data/single_sample_from_filtering/20250321_spe_sub_sampleID.rds"


In [ ]:
# 2.1 then process each sample separately for hvg and nnsvg via slurm job using following script (has the following steps in 2_end_stage_hvg_nnsvg_singlesamplejob.r)

# note after separating out individual samples, SITSA1 and SITSD3 have rows with 0 count (causing nnsvg to fail) for:
#SITSA1 - HNRNPCL3, TMOD4, REG1A USP17L10, PVRIG CNTNAP3B, ZP1, C13orf46 ZP2, PAGE2, TGIF2LY, PCDH11Y 
#SITSD3 - HNRNPCL3, PRAMEF33, UBE2U, CA14 AL590560.2, WDR64, SPDYA, NMS, ZDHHC19, USP17L10, HTN3, PCBD2, HIST1H4E, PVRIG 
         #ASZ1, AKR1C4, OOSP1, ZP1, DEFB131B, C13orf46, GOLGA6L6, TEPP, ALOX15, ANKRD30B, ZNF541, BPIFB2, BPIFB6, BPIFB3 
         #USP41, CRYBB2, GAGE1, PAGE2, SLC25A5, TGIF2LY, PRKY 
#so the following is run again within the script

# remove zero expression genes
ix_zero_genes <- rowSums(counts(spe_SITSD3)) == 0
table(ix_zero_genes)

# identify genes with 0 expression
true <- ix_zero_genes[ix_zero_genes]

if (sum(ix_zero_genes) > 0) {
  spe_SITSD3 <- spe_SITSD3[!ix_zero_genes, ]}

# remove spots with zero expression
ix_zero_spots <- colSums(counts(spe_SITSA1)) == 0
table(ix_zero_spots) #0

In [ ]:
#2.1 
#make sure you change which line is commented out in submit_spatial_scripts.sh so 2_end_stage_hvg_nnsvg_singlesamplejob.r is submitted

for sample in SITSA1 SITSA2 SITSA3 SITSA4 SITSB1 SITSB2 SITSB3 SITSB4 SITSC1 SITSC2 SITSC3 SITSC4 SITSD1 SITSD2 SITSD3 SITSD4 SITSE2 SITSE4 SITSF2 SITSF4 SITSG2 SITSH2; 
do     sbatch ../scripts/submit_spatial_scripts.sh $sample; done

2_end_stage_hvg_nnsvg_singlesamplejob.r:  
 - Loads the sample-specific RDS file.  
 - Removes genes and spots with zero expression.  
 - Applies a gene filter function (filter_genes) without mitochondrial filtering.  
 - Normalizes counts and computes library size factors.  
 - Identifies highly variable genes (HVGs) using modelGeneVar and flags them. Saves intermediate data objects.  
 - Checks for normalized log-counts, then runs the nnSVG spatial gene variability method.  
 - Saves the final object with nnSVG results.

2.3 then open interactive R session and proceed as follows (skip)
We will take the top ranking HVGs and top ranking SVGs, reduce the dimensions with PCA on each and compare  
 - runPCA() and runUMAP() from Seurat, by default calculates the top 20 PCs
 - Note results of comparing top HVGs and top SVGs are detailed below, for majority of samples top HVGs are contained within SVGs so use these for clustering and GO analysis 
 - Therefore skip HVG section below and go to section 2.4


In [ ]:
#2.3 fitting top HVGs

#knitr::opts_chunk$set(echo = TRUE)
library(spatialLIBD)
library(nnSVG)
library(SpatialExperiment)  
library(scater)
library(AnnotationHub)
library(tidyverse)
library(ggspavis)
library(scran)
library(DT)
library(HDF5Array)
projDir = "/mnt/scratchc/fmlab/lythgo02/visium_data/"



Load Pre-processed Samples:  
List files in a directory matching "spe_sub_modelgenevar" (these contain samples processed for highly variable genes, HVGs).  
Loop through the files, load each .rds file into R, and store the loaded data into spe_list (a list of SpatialExperiment objects).  


In [ ]:
#2.3
# ====== Load HVG-processed samples ======
load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/" 
# Alternative path (uncomment if needed)
# load_all_data <- "/mnt/nas-data/fmlab/group_folders/lythgo02/visium_data/single_sample_from_filtering/intermediate_results" 

# Sample IDs
samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", "SITSC4", 
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")  

# Load HVG-filtered RDS files
hvg_files <- list.files(path = load_all_data, pattern = "spe_sub_modelgenevar", full.names = TRUE)
names(hvg_files) <- samplesheet

# Load HVG objects into a list
spe_list <- list()
for (sample_name in names(hvg_files)) {
  spe <- readRDS(hvg_files[[sample_name]])
  spe_list[[sample_name]] <- spe
  cat("Loaded HVG sample:", sample_name, "\n")
}

Dimensionality reduction on top HVGs  
Top HVGs have been determined using getTopHVGs() from scran which returns top 10% (prop=0.1) of genes with highest biological variance after discarding those with bio <= 0  
Extract HVGs and Run PCA & UMAP (on PCAs) and save back to list  
  
Use PCA to reduce the dimensions of our dataset to assist clustering and UMAP to further reduce the principal components (PCs) in a two-dimensional space and produce better visualisations for the PCA.  
runPCA() function runs PCA on a SCE object, and returns an updated version of the single cell object with the PCA result added to the reducedDim slot.  

In [ ]:

#2.3
# ====== Extract HVGs and run PCA/UMAP ======
set.seed(123)

# Store HVG gene symbols
hvg <- list()
for (sample_name in names(spe_list)) {
  hvg[[sample_name]] <- rowData(spe_list[[sample_name]]) %>%
    as.data.frame() %>%
    filter(hvg == TRUE) %>%
    pull(symbol)

  # PCA
  spe_list[[sample_name]] <- runPCA(spe_list[[sample_name]], subset_row = hvg[[sample_name]], name = "PCA_hvg")
}

# Run UMAP on top PCs
set.seed(987)
for (sample_name in names(spe_list)) {
  spe_list[[sample_name]] <- runUMAP(spe_list[[sample_name]], dimred = "PCA_hvg", name = "UMAP_hvg")
  colnames(reducedDim(spe_list[[sample_name]], "UMAP_hvg")) <- c("UMAP_hvg1", "UMAP_hvg2")
}

# (Optional) Plot PCA
plots <- list()
for (sample_name in names(spe_list)) {
  sample_data <- spe_list[[sample_name]]
  pca_df <- as.data.frame(reducedDim(sample_data, "PCA_hvg"))
  pca_df$layer <- colData(sample_data)$ground_truth

  plots[[sample_name]] <- ggplot(pca_df, aes(x = PC1, y = PC2, colour = layer)) +
    geom_point(size = 0.5) +
    scale_colour_brewer(type = "qual") +
    labs(title = paste("PCA (Sample:", sample_name, ")"),
         x = "PC1", y = "PC2", colour = "layer") +
    theme_classic()
}

# Define the directory where you want to save the plots
#save_dir <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"

#for (i in seq_along(plots)) {
  # Define the file name based on the sample_id for uniqueness, with the full path
  #file_name <- paste0(save_dir, "plots/plot_hvg_pca", spe_list[[i]]$sample_id[1], ".png")
  
  # Save the plot using ggsave
 # ggsave(filename = file_name, plot = plots[[i]], width = 8, height = 6)
#}


2.4 Get top SVGs  - can use R markdown for this section also 

nnsvg  
The results are stored in the rowData of the SpatialExperiment object. The main results of interest are:  
 - LR_stat: likelihood ratio (LR) statistics used to rank SVGs  
 - rank: rank of top SVGs according to LR statistics  
 - pval: approximate p-values  
 - padj: approximate p-values adjusted for multiple testing  
 - prop_sv: effect size defined as proportion of spatial variance  
  
First load each spe object after having run nnsvg on the cluster  

In [ ]:
#2.4 
# ====== Load nnSVG-ranked samples ======
load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/" 
# Alternative path (uncomment if needed)
samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", "SITSC4", 
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")  
               
                 
svg_files <- list.files(path = load_all_data, pattern = "spe_sub_nnSVG", full.names = TRUE)
names(svg_files) <- samplesheet

spe_list_nnsvg <- list()
for (sample_name in names(svg_files)) {
  spe_svg <- readRDS(svg_files[[sample_name]])
  spe_list_nnsvg[[sample_name]] <- spe_svg
  cat("Loaded nnSVG sample:", sample_name, "\n")
}



Run PCA and UMAP on top SVGs  
 - take top 10% SVGs per sample for input to runPCA, in the example from https://bookdown.org/sjcockell/ismb-tutorial-2023/practical-session-2.html#inter-cluster-differentially-expressed-genes-dges they use top 10% HVGs = ~@1400 genes

In [ ]:
set.seed(456)

svg <- list()
common_genes_list <- list()

for (sample_name in names(spe_list_nnsvg)) {
  spe_svg <- spe_list_nnsvg[[sample_name]]
  spe_hvg <- spe_list[[sample_name]]

  # Compute top 10% SVGs
  n_top <- ceiling(0.10 * nrow(spe_svg))  # 10% of genes
  rowData(spe_svg)$svg <- rowData(spe_svg)$rank <= n_top

  # Extract SVG gene IDs
  svg[[sample_name]] <- rownames(spe_svg)[rowData(spe_svg)$svg]

  # PCA and UMAP on selected SVGs
  spe_svg <- runPCA(spe_svg, subset_row = svg[[sample_name]], name = "PCA_svg")
  spe_svg <- runUMAP(spe_svg, dimred = "PCA_svg", name = "UMAP_svg")
  colnames(reducedDim(spe_svg, "UMAP_svg")) <- c("UMAP_svg1", "UMAP_svg2")

  # Update object
  spe_list_nnsvg[[sample_name]] <- spe_svg

  # Find overlap with HVGs
 common_genes_list[[sample_name]] <- intersect(hvg[[sample_name]], svg[[sample_name]])
}


In [ ]:

for (sample in names(spe_list_nnsvg)){
    spe <- spe_list_nnsvg[[sample]]
    spe <- DelayedArray::realize(spe)
    saveRDS(spe,
    file=file.path(load_all_data, paste0(sample, "nnsvg_pca_umap.rds")))
}

ERROR: Error: object 'spe_nnsvg_list' not found


In [ ]:

#metadata table
group_df <- data.frame(
  sample_id = c(
    "SITSC1", "SITSD2", "SITSH2", "SITSC2", "SITSA3",
    "SITSA1", "SITSD4", "SITSG2", "SITSB3",
    "SITSB1", "SITSD1", "SITSA4", "SITSB4", "SITSF2",
    "SITSC3", "SITSF4", "SITSE4",
    "SITSB2",
    "SITSD3",
    "SITSC4", "SITSA2", "SITSE2"
  ),
    subgroup = c(
    rep("gBRCA1m-ov", 5),
    rep("gBRCA2m-ov", 4),
    rep("HRP-ov", 5),
    rep("non-BRCA-HRD-ov", 3),
    "gBRCA2m-om",
    "gBRCA1m-om",
    rep("HRP-om", 3)
  )
)

group_df$genotype <- sub("-o.*", "", group_df$subgroup)


gene_summary <- data.frame(
  sample_id=group_df$sample_id,
  subgroup=group_df$subgroup,
  hvg_n=sapply(group_df$sample_id, function(sample)
  length(hvg[[sample]])),
  hvg_in_svg_n=sapply(group_df$sample, function(sample)
  length(common_genes_list[[sample]])))

gene_summary$genotype <- sub("-o.*", "", gene_summary$subgroup)

gene_summary$proportion <- gene_summary$hvg_in_svg_n / gene_summary$hvg_n



# Set output directory
save_dir <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/plots/"
write.csv(gene_summary, paste0(save_dir, "hvg_svg_gene_summary.csv"))



In [ ]:

# 1️⃣ Boxplot
p1 <- ggplot(gene_summary, aes(x = genotype, y = proportion, fill = genotype)) +
  geom_boxplot(outlier.shape = NA, alpha = 0.5) +
  geom_jitter(width = 0.2, size = 2) +
  labs(
    title = "Proportion of HVGs also found in SVGs",
    x = "Genotype",
    y = "Proportion HVGs in SVGs"
  ) +
  theme_minimal(base_size = 14) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  scale_fill_brewer(palette = "Set2")

# Save boxplot
ggsave(filename = file.path(save_dir, "hvg_svg_proportion_boxplot.png"), plot = p1, width = 8, height = 6)

# 2️⃣ Per-sample line plot
p2 <- ggplot(gene_summary, aes(x = reorder(sample_id, proportion), y = proportion, color = genotype)) +
  geom_point(size = 3) +
  geom_segment(aes(xend = reorder(sample_id, proportion), y = 0, yend = proportion), alpha = 0.4) +
  labs(
    title = "Per-sample HVG-SVG Proportion",
    x = "Sample ID",
    y = "Proportion"
  ) +
  theme_minimal(base_size = 13) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)) +
  scale_color_brewer(palette = "Set2")

# Save per-sample plot
ggsave(filename = file.path(save_dir, "hvg_svg_proportion_by_sample.png"), plot = p2, width = 10, height = 6)

Gene variability:
 - HVGs = Highly Variable Genes: These are genes showing high variation across spots/samples.
 - SVGs = Spatially Variable Genes: Genes whose expression varies with spatial location (spots).
 - Overlap (hvg_in_svg_n): Number of HVGs that are also spatially variable in that sample.

Biological / Analytical insights:
 - High Proportion (~0.7-1.0): Most HVGs in these samples are spatially variable, implying that gene variability is largely driven by spatial differences in tissue.
 - Example: SITSA1 has a perfect 1.0 proportion — all its HVGs are spatially variable.
 - This suggests spatial patterns strongly influence gene variability here.
 - Lower Proportion (~0.45-0.6): A lower fraction of HVGs are spatially variable.
 - Example: SITSD4 with 0.45 proportion means more than half HVGs are variable but not necessarily spatially patterned — possibly due to cell type variation or technical noise.
 - Inter-sample variability: The proportion varies between samples, which might reflect biological differences (e.g., different tissue regions, pathological states) or technical differences (e.g., sequencing depth).

SVGs capture genes whose expression varies spatially, which can reveal tissue structure and microenvironment = primary focus so focussing on these (esp given that for most samples >70% HVGs are in SVGs)



Integrate SVGs and HVGs (or just take the SVGs if the HVGs are in SVGs for each sample)........  
I looked into the following statement/paper and it doesn't actually seem to comment on SVG/HVG integration  
A recent benchmark paper (Li et al. 2022) showed that integrating HVGs and SVGs to generate a combined set of features can improve downstream clustering performance in STx data. This confirms that SVGs contain additional biologically relevant information that is not captured by HVGs in these datasets. For example, a simple way to combine these features is to concatenate columns of principal components (PCs) calculated on the set of HVGs and the set of SVGs (excluding overlapping HVGs), and then using the combined set of features for further downstream analyses (Li et al. 2022).

Cluster gene expression via scRNAseq methods:
 - Graph-based clustering using the Walktrap method implemented in scran (Lun, McCarthy, and Marioni 2016), builds shared nearest neighbour (SNN) using top 50 PCs calculated on the set of top SVGs.
 - Assume that biologically informative spatial distribution patterns of cell types can be detected from the molecular features (gene expression).
 - The chunk below clusters without taking into account xy coordinates (may be better at picking up niche relationships but trying with the spatial info accounted for initially using chunk after this)


In [ ]:
set.seed(123)
cluster_summary <- list()

for (sample in names(spe_list_nnsvg)) {
  sce <- spe_list_nnsvg[[sample]]

  # Build SNN using PCs from nnSVG
  g <- scran::buildSNNGraph(
    sce,
    k = 35,
    use.dimred = "PCA_svg",
    type = "jaccard"
  )
  # Adjust resolution for granularity (lower = fewer clusters)
  res <- if (ncol(sce) < 1500) 0.35 else 0.5

    # --- Sample-specific overrides ---
  if (sample %in% c("SITSA1","SITSB2", "SITSC3", "SITSC1", "SITSC4", "SITSE4", "SITSD4", "SITSH2")) {
    res <- 0.35
  }
  if (sample %in% c("SITSA4", "SITSC2")) {
    res <- 0.3
  }

  # Leiden clustering
  g_leiden <- igraph::cluster_leiden(
    g,
    objective_function   = "modularity",     # or "CPM" for finer structure
    weights              = igraph::E(g)$weight,
    resolution_parameter = res,
    n_iterations         = -1
  )
  clus <- g_leiden$membership
  # Store cluster labels and summaries
  colLabels(sce) <- factor(clus)
  cluster_summary[[sample]] <- table(clus)
  spe_list_nnsvg[[sample]] <- sce
}



The approach described in the tutorial then suggests adding the raw (or normalized) x-y spatial coordinates as additional dimensions during clustering — concatenated with SVG PCAs 
This directly and explicitly forces the clustering to also consider physical proximity in space.

The following section combines PCAs and xy-coordinates as a slot in the spatial experiment named "PCA_svg_plus_xy"  
NOT DOING THIS because the cluster n that results are too high and granular 

In [ ]:

#For example:

# Combine PCA of SVGs + spatial coordinates
#spot_counts <- sapply(spe_list_nnsvg, ncol)
#spot_counts_df <- data.frame(spot_counts, 
                             guide_k = ifelse(spot_counts>2000,15,10)) # if spot number >2000 k = 15, otherwise k=10


#for (sample in names(spe_list_nnsvg)) {
#  spe <- spe_list_nnsvg[[sample]]  # extract sample object
  # Extract PCA
#  pcs <- reducedDim(spe, "PCA_svg")
  # Extract and relabel coords to match colnames (with sample ID)
#  coords <- spatialCoords(spe)
#  rownames(coords) <- paste0(sub("-1$", "", rownames(coords)), "-", sample)
  # Sanity check: ensure rownames match
#  stopifnot(identical(rownames(pcs), rownames(coords)))
  # Combine PCA + spatial coordinates
#  combined <- cbind(pcs, coords)
#  reducedDim(spe, "PCA_svg_plus_xy") <- combined
  # Set k from spot_counts_df (ensure rownames are correct)
#  k_value <- spot_counts_df[sample, "guide_k"]
  # Build SNN graph using reducedDim
#  g <- buildSNNGraph(spe, k = 15, use.dimred = "PCA_svg_plus_xy")
  # Apply Walktrap clustering
#  g_walk <- igraph::cluster_walktrap(g)
#  clus <- g_walk$membership
  # Assign clusters
#  colLabels(spe) <- factor(clus)
  # Save updated object back into list
#  spe_list_nnsvg[[sample]] <- spe
#}


In [ ]:
save_dir <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"
#for each sample, count how many unique clusters were found
cluster_counts <- data.frame(
  sample_id = names(spe_list_nnsvg),
  n_clusters = sapply(spe_list_nnsvg, function(s) length(unique(colLabels(s)))),
  n_spots=sapply(spe_list_nnsvg, ncol)
)
cluster_counts
#add meta data
cluster_counts$subgroup <- group_df$subgroup[match(cluster_counts$sample_id, group_df$sample_id)]
cluster_counts$genotype <- sub("-o.*", "", cluster_counts$subgroup)

write.csv(cluster_counts, paste0(save_dir, "cluster_counts.csv"), row.names = FALSE)

library(ggplot2)
#plot cluster number per sample, coloured by genotype
p3 <-ggplot(cluster_counts, aes(x = reorder(sample_id, n_clusters), y = n_clusters, color = genotype)) +
  geom_point(size = 3) +
  geom_segment(aes(xend = reorder(sample_id, n_clusters), y = 0, yend = n_clusters), alpha = 0.4) +
  labs(
    title = "Per-sample Cluster Diversity",
    x = "Sample ID",
    y = "Number of Clusters"
  ) +
  theme_minimal(base_size = 13) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)) +
  scale_color_brewer(palette = "Set2")

p4 <- ggplot(cluster_counts, aes(x = n_spots, y = n_clusters, color = subgroup, label = sample_id)) +
  geom_point(size = 3) +
  geom_text(vjust = -0.8, size = 3) +
  theme_minimal() +
  labs(
    title = "Number of clusters vs number of spots per sample",
    x = "Number of spots",
    y = "Number of clusters"
  ) +
  theme(plot.title = element_text(face = "bold", hjust = 0.5))

ggsave(filename = file.path(save_dir, "spatial_clusters.png"), plot = p3, width = 10, height = 6)
ggsave(filename = file.path(save_dir, "spotsvsclusters.png"), plot = p4, width = 10, height = 6)

Plot spatial domains  
Plot PCA coloured by spatial domain label  
Plot UMAP coloured by spatial domain label  

In [ ]:
results <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"

spe_list_nnsvg <- readRDS(file.path(results, "spe_list_nnsvg_clusters.rds"))

: 

In [ ]:

for (sample in names(spe_list_nnsvg)) {
  spe <- spe_list_nnsvg[[sample]]
  plotSpots(spe, annotate = "label")
  ggsave(filename =  file.path(save_dir, paste0(sample, "_SVGs_clustered.png")), width = 6, height = 6)
  plotDimRed(spe, plot_type="UMAP_svg", annotate="label")
  ggsave(filename =  file.path(save_dir, paste0(sample, "_umap.png")), width = 6, height = 6)
  #plotDimRed(spe, plot_type="PCA_svg", annotate="label")
  #ggsave(filename =  file.path(save_dir, paste0(sample, "_pca.png")), width = 6, height = 6)
}


Next steps
Differential Expression Between Clusters
Gene Ontology / Pathway Enrichment

Engineer towards Aim 2: Characterize gene expression programs underpinning immune cell states
•	Identify transcriptional programs (e.g., exhaustion, cytotoxicity, IFN response, chemokine signaling) that define functional immune states and differ by genomic subtype.
o	Focus: Cell-intrinsic functional states and pathway enrichment
o	BRCA1 tumours → high IFN/CXCL9/10 → exhausted T cells; BRCA2 → less IFN but more NK cell infiltration. Diversity amongst non BRCA HRD and HRP which is yet to be defined.
o	Goal: Link T, NK and tumour cell behaviors and pathways to genomic context 


Perform gene ontology enrichment abnalysis on cluster-specific DGEs:  
For each sample, and each cluster within the sample, perform:  
    Differential expression (cluster vs all others)  
    Select DEGs (FDR < 0.05)  
    Map gene symbols → Entrez IDs  
    Run GO enrichment analysis (Biological Process category)  

In [ ]:
#2.5 
# ====== Load clusstering results ======
load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/" 
# Alternative path (uncomment if needed)
samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", "SITSC4", 
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")  
                 
svg_files <- list.files(path = load_all_data, pattern = "nnsvg_pca_umap.rds", full.names = TRUE)
names(svg_files) <- samplesheet

spe_list_nnsvg <- list()
for (sample_name in names(svg_files)) {
  spe_svg <- readRDS(svg_files[[sample_name]])
  spe_list_nnsvg[[sample_name]] <- spe_svg
  cat("Loaded nnSVG sample:", sample_name, "\n")
}


In [ ]:
results <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"

spe_list_nnsvg <- readRDS(file.path(results, "spe_list_nnsvg_clusters.rds"))

Extract gene list per cluster for each sample and find DE genes between clusters

In [ ]:
library(clusterProfiler)
library(org.Hs.eg.db)
library(scater)


de_results_list <- list()

for (sample in names(spe_list_nnsvg)) {
  spe <- spe_list_nnsvg[[sample]]
  clusters <- colLabels(spe)

  # DE between clusters
  de <- findMarkers(spe, groups = clusters, test.type = "t",  lfc = 0.5, direction = "up")
  de_results_list[[sample]] <- de 
   # ALSO save into the SCE object itself
  metadata(spe)$de_markers <- de

  # Put updated object back into the list
  spe_list_nnsvg[[sample]] <- spe 
  }

In [ ]:
saveRDS(spe_list_nnsvg, paste0(results, "spe_list_nnsvg_clusters_DEmarkers.rds"))

To load spe list with DE markers (raw unfiltered)

In [ ]:
results <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"

spe_list_nnsvg <- readRDS(file.path(results, "spe_list_nnsvg_clusters_DEmarkers.rds"))

In [ ]:
# -------------------------------------------------------------------------
# Extract filtered DE marker genes per cluster per sample
# Criteria:
#   - FDR < 0.05  (statistical significance)
#   - summary.logFC >= 1  (minimum effect size ≈ 2-fold change)
# -------------------------------------------------------------------------
# Export filtered DE tables per sample with full statistics
# -------------------------------------------------------------------------
output_dir <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/de_csv/"
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

for (sample in names(de_results_list)) {

  de <- de_results_list[[sample]]

  # Build one long dataframe per sample (stacking clusters)
  sample_df <- dplyr::bind_rows(lapply(names(de), function(cluster_id) {

    df <- as.data.frame(de[[cluster_id]])

    # Apply your filtering criteria
    keep <- df$FDR < 0.05 & df$summary.logFC >= 1
    df <- df[keep, , drop = FALSE]

    # If nothing passes, return NULL so bind_rows skips it
    if (nrow(df) == 0) return(NULL)

    # Add identifiers
    df$gene <- rownames(df)
    df$cluster <- cluster_id
    df$sample <- sample

    # Reorder so identifiers come first
    df <- df[, c("sample", "cluster", "gene",
                 setdiff(colnames(df), c("sample", "cluster", "gene"))),
             drop = FALSE]

    df
  }))

  # If no clusters had any passing genes, skip writing
  if (is.null(sample_df) || nrow(sample_df) == 0) next

  # Write per-sample CSV
  write.csv(
    sample_df,
    file.path(output_dir, paste0(sample, "_DE_top_hits.csv")),
    row.names = FALSE
  )
}


Map DEGs to Entrez IDs

In [ ]:
entrez_list <- list()

for (sample in names(de_results_list)) {
  de <- de_results_list[[sample]]
  sample_entrez <- list()

  for (cluster_id in names(de)) {

    top_genes <- rownames(de[[cluster_id]])[
      de[[cluster_id]]$FDR < 0.05 & de[[cluster_id]]$summary.logFC >= 1 #filter by FDR and effect size (logFC) to reduce gene number
    ]
    top_genes <- unique(na.omit(top_genes))
    if (length(top_genes) < 5) next

    gene_df <- bitr(top_genes, fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Hs.eg.db)
    if (is.null(gene_df) || nrow(gene_df) < 5) next

    entrez <- unique(na.omit(gene_df$ENTREZID))
    if (length(entrez) < 5) next

    sample_entrez[[cluster_id]] <- entrez
  }

  if (length(sample_entrez) > 0) {
    entrez_list[[sample]] <- sample_entrez
  }
}


Check degree of overlap in marker genes between clusters  
 - if multiple clusters in a sample exceed 0.6-0.8 shared DE genes between clusters then suggests over-clustered, adjust resolution and k accordingly
 - used to set sample specific resolution for clustering 

In [ ]:
#check overlap in DE genes distinguishing clusters
overlap_proportion <- lapply(entrez_list, function(sample_clusters) {
  cluster_ids <- names(sample_clusters)
  n_clusters  <- length(cluster_ids)
  
  # Matrix to store pairwise overlap proportions
  overlap_matrix <- matrix(NA, n_clusters, n_clusters,
                           dimnames = list(cluster_ids, cluster_ids))
  
  for (cluster_i in cluster_ids) {
    genes_i <- sample_clusters[[cluster_i]]
    
    for (cluster_j in cluster_ids) {
      genes_j <- sample_clusters[[cluster_j]]
      
      shared_genes <- length(intersect(genes_i, genes_j))
      total_genes_i <- length(genes_i)
      
      overlap_matrix[cluster_i, cluster_j] <- shared_genes / total_genes_i
    }
  }
  
  overlap_matrix
})



In [ ]:
results <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"

spe_list_nnsvg <- readRDS(file.path(results, "spe_list_nnsvg_clusters.rds"))

clusterProfiler https://pmc.ncbi.nlm.nih.gov/articles/PMC8454663/

GO enrichment analysis 
 - use enrichGO() from clusterProfiler() 
 - Categorises genes into chosen ontology (biological processes BP) using GO consortium to give list of enriched biological processes 

Performs gene ontology over-representation (ORA GO) test by hypergeometric test (essentially Fisher's exact test) to test whether GO term is enriched in cluster/subgroup genes compared to background
 - GO terms are organised as a graph with edges representing a parent-child relationship, so parent terms and all of it's child terms will be returned in GO results, hence there is a lot of redundancy in the list of GO terms returned  
 - There, we run simplify() after which uses GOSemSim package to calculate semantic similarity between terms are remove redundant terms 

In [ ]:
compare_go_list <- list()

for (sample in names(entrez_list)) {
  message("Running compareCluster (GO) for ", sample, "...")

  compare_go_list[[sample]] <- compareCluster(
    geneCluster   = entrez_list[[sample]],
    fun           = "enrichGO",
    ont           = "BP",
    OrgDb         = org.Hs.eg.db,
    pAdjustMethod = "BH",
    qvalueCutoff  = 0.05
  )
}

In [ ]:
Remove redundancy in terms using simplify() from clusterprofiler
 - amongst similar terms, it picks the most statistically significant term

In [ ]:
go_compare_simplified <- lapply(compare_go_list, simplify)

KEGG pathway analysis 
 - Pathway based interpretation of how genes work together in molecular network using Kyoto Encyclopedia of Genes and Genomes (KEGG) curated database of pathways
 - molecular mechanisms/signalling interpretation 

Or run withh compareCluster  
 - calculates enrichment between clusters and aggregates into single object

In [ ]:
compare_kegg_list <- list()

for (sample in names(entrez_list)) {
  message("Running compareCluster (KEGG) for ", sample, "...")

  compare_kegg_list[[sample]] <- compareCluster(
    geneCluster   = entrez_list[[sample]],
    fun           = "enrichKEGG",
    organism      = "hsa",
    pAdjustMethod = "BH",
    qvalueCutoff  = 0.05
  )
}

Reactome analysis  
 - additional pathway based database (more detailed and hierarchical than KEGG)
 - human specific (unlike KEGG)
  

In [ ]:
library(ReactomePA)  

compare_reactome_list <- list()
for (sample in names(entrez_list)) {
  compare_reactome_list[[sample]] <- compareCluster(
    geneCluster   = entrez_list[[sample]],
    fun           = "enrichPathway",  # qualify the function
    organism      = "human",                      # NOT "hsa"
    pAdjustMethod = "BH",
    pvalueCutoff  = 0.05
  ) 
}

MSigDB analysis for Hallmark gene pathways 
MSigDB Hallmark gene sets represent a compact collection of core biological and tumour programs reflecting cellular states such as inflammation, stromal activation, proliferation, metabolism, and immune signaling.
https://pmc.ncbi.nlm.nih.gov/articles/PMC4707969/

In [ ]:
library(msigdbr)
library(clusterProfiler)
library(dplyr)

# MSigDB Hallmark gene sets (human)
m_df <- msigdbr(species = "Homo sapiens", category = "H")

# TERM2GENE object
m_t2g <- m_df %>%
  dplyr::select(gs_name, entrez_gene)

In [ ]:
compare_msigdb_list <- list()

for (sample in names(entrez_list)) {
  message("Running compareCluster (MSigDB) for ", sample, "...")

  compare_msigdb_list[[sample]] <- compareCluster(
    geneCluster   = entrez_list[[sample]],
    fun           = "enricher",
    TERM2GENE     = m_t2g,
    pAdjustMethod = "BH",
    qvalueCutoff  = 0.05
  )
}


In [ ]:
results <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/no_mito/"
#saveRDS(kegg_results_list, paste0(results, "kegg_results.rds"))
saveRDS(compare_kegg_list, paste0(results, "kegg_clustercompare_results.rds"))
saveRDS(go_compare_simplified, paste0(results, "clustercompare_go_results_simplified.rds"))
saveRDS(compare_reactome_list, paste0(results, "reactome_comparecluster_results.rds"))
saveRDS(spe_list_nnsvg, paste0(results, "spe_list_nnsvg_clusters.rds"))
saveRDS(compare_msigdb_list, paste0(results, "msigdb_comparecluster_results.rds"))

In [ ]:
Subgroup comparison to try and narrow the scope of investigations into cluster heterogeneity 
 - pool DE genes (from clusters) per subgroup, take genes that are consistently DE (>= 2 samples) and perform enrichment analyses 

In [ ]:

immune_lookup <- tribble(
  ~sits_id, ~immune_cat,
  "SITSB2", "Effector dominant",
  "SITSA4", "Macrophage dominant",
  "SITSB1", "Effector dominant",
  "SITSC1", "Immune Desert",
  "SITSB4", "Immune Desert",
  "SITSA3", "Macrophage dominant",
  "SITSD4", "Immune Desert",
  "SITSA2", "Immune Desert",
  "SITSC2", "Effector dominant",
  "SITSD3", "Effector dominant",
  "SITSG2", "Macrophage dominant",
  "SITSD2", "Macrophage dominant",
  "SITSH2", "Macrophage dominant",
  "SITSE4", "Immune Desert",
  "SITSC3", "Macrophage dominant",
  "SITSF4", "Macrophage dominant",
  "SITSE2", "Effector dominant",
  "SITSF2", "Immune Desert",
  "SITSA1", "Immune Desert",
  "SITSB3", "Immune Desert",
  "SITSC4", "Immune Desert",
  "SITSD1", "Immune Desert"
)

Pool genes per subgroup, two options:
 - chunk directly below pools genes and then applies a filter so that only genes present in > 30% samples (in the subgroup) are kept 
 - chunk that follows (in use) just takes all genes and counts each gene only once (regardless of number of clusters or samples the gene is found in)
There are various arguments for whihc way to do it but each has it's own biases and it's ok if aware of limitations 

In [ ]:

#genes_per_subgroup <- list()

#min_prop <- 0.30  # adjust to 0.4 if you want stricter

#for (subgrp in unique(immune_lookup$immune_cat)) {

 # samples <- immune_lookup$sits_id[immune_lookup$immune_cat == subgrp]
 # n_samp  <- length(samples)
 # min_n   <- max(2, ceiling(min_prop * n_samp))  # size-controlled threshold

  # collect genes per sample (unique within sample)
  #sample_genes <- lapply(samples, function(s) {
  #  if (!is.null(entrez_list[[s]])) unique(unlist(entrez_list[[s]])) else character(0)
  #})

  # prevalence across samples (each sample contributes 0/1 per gene)
  #gene_counts <- table(unlist(sample_genes))

  #genes_per_subgroup[[subgrp]] <- names(gene_counts[gene_counts >= min_n])
#}


In [ ]:

#    (keep genes present in >=1 samples)  gives same result as present in >= 2 samples 
# -----------------------------
genes_per_subgroup <- list()

for (subgrp in unique(immune_lookup$immune_cat)) {

  samples <- immune_lookup$sits_id[immune_lookup$immune_cat == subgrp]

  # collect genes per sample (skip NULLs)
  sample_genes <- lapply(samples, function(s) {
    if (!is.null(entrez_list[[s]])) unlist(entrez_list[[s]]) else character(0)
  })

  # IMPORTANT: count presence across samples (not total occurrences)
  gene_counts <- table(unlist(lapply(sample_genes, unique)))

  genes_per_subgroup[[subgrp]] <- names(gene_counts[gene_counts >= 1])
}



ORA per subgroup - hallmark pathways

In [ ]:

# -----------------------------
# 2) ORA per subgroup + save CSV
# -----------------------------
ora_subgroup_results <- list()

for (subgrp in names(genes_per_subgroup)) {

  message("Running ORA for subgroup: ", subgrp)

  pooled_genes <- genes_per_subgroup[[subgrp]]

  if (length(pooled_genes) < 10) {
    message("Skipping ", subgrp, " (too few genes: ", length(pooled_genes), ")")
    next
  }

  res <- enricher(
    gene          = pooled_genes,
    TERM2GENE     = m_t2g,
    pAdjustMethod = "BH",
    pvalueCutoff  = 0.05,
    minGSSize     = 15,
    maxGSSize     = 500
  )

  if (is.null(res) || nrow(as.data.frame(res)) == 0) next

  ora_subgroup_results[[subgrp]] <- res

  write.csv(
    as.data.frame(res),
    file = file.path(results, paste0("ora_subgroup_", gsub(" ", "_", subgrp), "_msigdb_hallmark.csv")),
   # file = file.path(results, paste0("ora_subgroup_", gsub(" ", "_", subgrp), "_msigdb_immune_C7.csv")),
    row.names = FALSE
  )
}


In [ ]:
Subgroup comparison using immune set of genes from MSigDB (C7)

In [ ]:
# MSigDB Hallmark gene sets (human)
m_df <- msigdbr(species = "Homo sapiens", category = "C7")

# TERM2GENE object
m_t2g <- m_df %>%
  dplyr::select(gs_name, entrez_gene)

  #then re-run the code chunk above

KEGG per subgroup 

In [ ]:

ora_kegg_subgroup <- list()

for (subgrp in names(genes_per_subgroup)) {

  message("Running KEGG ORA for subgroup: ", subgrp)

  pooled_entrez <- unique(na.omit(genes_per_subgroup[[subgrp]]))

  if (length(pooled_entrez) < 10) {
    message("Skipping ", subgrp, " (too few genes: ", length(pooled_entrez), ")")
    next
  }

  res <- enrichKEGG(
    gene          = pooled_entrez,
    organism      = "hsa",
    pAdjustMethod = "BH",
    pvalueCutoff  = 0.05,
    qvalueCutoff  = 0.05,
    minGSSize     = 15,
    maxGSSize     = 500
  )

  if (is.null(res) || nrow(as.data.frame(res)) == 0) next

  # Optional: convert Entrez IDs in the result to readable gene symbols
  res <- setReadable(res, OrgDb = org.Hs.eg.db, keyType = "ENTREZID")

  ora_kegg_subgroup[[subgrp]] <- res

  write.csv(
    as.data.frame(res),
    file = file.path(results, paste0("ora_subgroup_", gsub(" ", "_", subgrp), "_KEGG.csv")),
    row.names = FALSE
  )
}

Genotype specific subgroup GO comparison 

In [ ]:

#or for genotype specific


#metadata table
group_df <- data.frame(
  sample_id = c(
    "SITSC1", "SITSD2", "SITSH2", "SITSC2", "SITSA3",
    "SITSA1", "SITSD4", "SITSG2", "SITSB3",
    "SITSB1", "SITSD1", "SITSA4", "SITSB4", "SITSF2",
    "SITSC3", "SITSF4", "SITSE4",
    "SITSB2",
    "SITSD3",
    "SITSC4", "SITSA2", "SITSE2"
  ),
    subgroup = c(
    rep("gBRCA1m-ov", 5),
    rep("gBRCA2m-ov", 4),
    rep("HRP-ov", 5),
    rep("non-BRCA-HRD-ov", 3),
    "gBRCA2m-om",
    "gBRCA1m-om",
    rep("HRP-om", 3)
  )
)

group_df$genotype <- sub("-o.*", "", group_df$subgroup)

#get genes per subgroup
#    (keep genes present in >=2 samples)
# -----------------------------
genes_per_subgroup <- list()

for (subgrp in unique(group_df$genotype)) {

  samples <- group_df$sample_id[group_df$genotype == subgrp]

  # collect genes per sample (skip NULLs)
  sample_genes <- lapply(samples, function(s) {
    if (!is.null(entrez_list[[s]])) unlist(entrez_list[[s]]) else character(0)
  })

  # IMPORTANT: count presence across samples (not total occurrences)
  gene_counts <- table(unlist(lapply(sample_genes, unique)))

  genes_per_subgroup[[subgrp]] <- names(gene_counts[gene_counts >= 1])
}

names(genes_per_subgroup) <- unique(group_df$genotype)

#or with size control (e.g. gene must be present in >=30% of samples in subgroup)

#min_prop <- 0.30  # set to 0.40 for stricter consistency

#genes_per_subgroup <- lapply(unique(group_df$genotype), function(subgrp) {
 # samples <- group_df$sample_id[group_df$genotype == subgrp]
 # n_samp  <- length(samples)

  # Size-controlled minimum number of samples a gene must appear in
  #min_n <- max(2, ceiling(min_prop * n_samp))

  # Collect DE genes per sample (unique within sample)
  #sample_genes <- lapply(samples, function(s) {
  #  if (!is.null(entrez_list[[s]])) unique(unlist(entrez_list[[s]])) else character(0)
  #})
  #gene_counts <- table(unlist(sample_genes))
  #names(gene_counts[gene_counts >= min_n])
#})

#names(genes_per_subgroup) <- unique(group_df$genotype)


In [ ]:
go_results_subgroup <- lapply(names(genes_per_subgroup), function(subgrp) {
  enrichGO(
    gene = genes_per_subgroup[[subgrp]],
    OrgDb = org.Hs.eg.db,
    ont = "BP",
    pAdjustMethod = "BH",
    pvalueCutoff = 0.05,
    readable = TRUE
  )
})

go_results_subgroup_simplified <- lapply(go_results_subgroup, function(ego) {
  simplify(
    ego,
    cutoff = 0.7,
    by = "p.adjust",
    select_fun = min
  )
})


In [ ]:

plots<- lapply(names(go_results_list), function(sample){
  sample_results <- go_results_list[[sample]]
  
  cluster_plots <- lapply(names(sample_results), function(cluster){
    cluster_result <- sample_results[[cluster]]
    p <- barplot(cluster_result, showCategory=5) +
      ggtitle(cluster)+
      theme( axis.text.x = element_text(size = 6, angle = 45, hjust = 1),  # smaller and rotated x-axis labels
        axis.text.y = element_text(size = 6),
        axis.title.x= element_text(size=6),
        plot.title = element_text(size = 6, face = "bold"),
         legend.title = element_text(size = 6),                         # legend title
        legend.text = element_text(size = 6) 
      )
    return(p)
 })
  names(cluster_plots) <- names(sample_results)
  return(cluster_plots)
})
names(plots) <- names(go_results_list)


In [ ]:
Plot results in playing

In [ ]:
library(HDF5Array)
#saved locally on workstation
dir.create("Home/Documents/saved_spe_h5", showWarnings = FALSE)

for (sample in names(spe_list_nnsvg[13:22])) {
  spe <- spe_list_nnsvg[[sample]]
  saveHDF5SummarizedExperiment(
    spe,
    dir = file.path("saved_spe_h5", paste0(sample, "_spe_nnsvg_clusterDE")),
    replace = TRUE
  )
}


# Load HVG-filtered RDS files
samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", "SITSC4", 
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")  
                 
cluster_files <- list.files(path = "/home/lythgo02/Documents/saved_spe_h5/", pattern = "_spe_nnsvg_clusterDE", full.names = TRUE)
spe_list <- list()
names(cluster_files) <- samplesheet

for (sample in names(cluster_files)){
  spe <- loadHDF5SummarizedExperiment(cluster_files[[sample]])
  spe_list[[sample]] <- spe
}


library(zellkonverter)

# Assuming your object is called `spe`

for (sample in names(spe_list_nnsvg[1:22])) {
  spe <- spe_list_nnsvg[[sample]]
  zellkonverter::writeH5AD(
    spe,
    file = file.path("/mnt/scratchc/fmlab/lythgo02/Spatial/saved_spe_h5ad", paste0(sample, "_spe_nnsvg_clusterDE.h5ad"))
  )
}

In [ ]:
Run clustering and GO enrichment within each sample (as you already do).

For each subgroup, summarize:
    Number of clusters per sample (diversity)
    Recurrent GO terms across clusters and samples
    Possibly score clusters by significance or similarity
    Identify patterns across samples in the same subgroup:
    Shared GO terms
    Shared cluster structures (e.g., similar cell states)
    Variability within/between subgroups

Use cluster locations (from colData/colLabels) to compare with image-derived features:
    In H&E: align clusters with morphology (e.g., necrosis, ductal regions, stroma).
    In mIF: are immune-rich clusters spatially aligned with CD3+, CD8+, or macrophage-rich zones?

Advantages:
Preserves spatial resolution and biological individuality of each sample.Handles sample heterogeneity well. Identifies dentify subgroup-specific recurrent patterns, not just averages